In [1]:
from compress_vid import *

testfile="./test/DJI_0021.MP4"
testdir="./test"

In [15]:
def get_video_info(input_path):
    # ffprobe command to get video codec, dimensions, bitrate, fps, duration, and file size
    cmd = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-show_entries', 'stream=codec_name,width,height,bit_rate,r_frame_rate',
        '-show_entries', 'format=duration,size',  # Including format=size to get the file size
        '-of', 'default=noprint_wrappers=1:nokey=1',
        input_path
    ]

    # Execute the ffprobe command
    output = subprocess.check_output(cmd).decode('utf-8').strip().split('\n')

    # Extracting the information from the output
    video_codec = output[0]
    width = output[1]
    height = output[2]
    bitrate = output[4]
    fps_frac = output[3]
    duration = float(output[5])
    file_size = output[6]  # Corrected file size index

    # Calculating FPS from the fractional representation
    fps = "N/A"
    if fps_frac:
        try:
            numerator, denominator = map(int, fps_frac.split('/'))
            fps = round(float(numerator) / float(denominator), 2)
        except ValueError:
            # Handle the case where the FPS is not a fraction (which is rare)
            fps = float(fps_frac)

    # Formatting dimensions
    dimensions = f"{width}x{height}"

    # Formatting bitrate as Mb/s if bitrate is available
    bitrate_mbps = "N/A"
    if bitrate and bitrate.isdigit():
        bitrate_mbps = round(int(bitrate) / 1e6, 1)

    # Formatting file size as MB
    size_mb = round(int(file_size) / 1e6, 1)

    # Formatting duration as HH:MM:SS
    duration_str = str(datetime.timedelta(seconds=duration))

     # Constructing the dictionary with the video information
    video_info = {
        'video_codec': video_codec,
        'dimensions': dimensions,
        'bitrate_mbps': bitrate_mbps,
        'fps': fps,
        'duration_str': duration_str,
        'size_mb': size_mb
    }

    # Returning the constructed dictionary
    return video_info

# get_video_info(testfile)

def get_rating(file_path):
    # Command to get the rating using exiftool
    rating_cmd = ['exiftool', '-XMP:Rating', file_path]

    # Run the command and capture the output
    completed_process = subprocess.run(rating_cmd, stdout=subprocess.PIPE, text=True)

    # Check if the command was successful
    if completed_process.returncode == 0:
        # Extract the rating from the output
        output = completed_process.stdout.strip()

        # Assume output format: "XMP:Rating                        : <RatingValue>"
        # Split the output and get the last element, which should be the rating
        rating = output.split(':')[-1].strip()

        return rating
    else:
        # Handle error if exiftool failed
        print("Error: exiftool did not complete successfully.")
        return None



def get_export_settings(input_path):
    # Use the get_video_info function to get video parameters
    video_info = get_video_info(input_path)
    dimensions = video_info['dimensions']
    fps = video_info['fps']
    codec = 'vt_h265'
    # Default quality
    quality = 'LQ'
    # Update quality if rating is 4 or higher
    if get_rating(input_path) >= 4:
        quality = 'HQ'
    
    # Determine resolution based on dimensions
    width, height = map(int, dimensions.split('x'))
    print(width, height)
    resolution = None
    if (width >= 3840 and height >= 2160) or (width >= 2160 and height >= 3840):  # Handling both landscape and portrait 4K
        resolution = '4k'
    elif (width >= 1920 and height >= 1080) or (width >= 1080 and height >= 1920):  # Handling both landscape and portrait 1080p
        resolution = '1080p'

    # Round fps to the nearest whole number to match against '30' or '60', 24 & 25 are considered 30
    if fps <= 30:
        frame_rate = '30'
    else:
        frame_rate = str(round(fps / 30) * 30)

    # Use resolution and frame rate to get the correct settings from the dictionary
    settings = ffmpeg_settings.get(resolution, {}).get(str(frame_rate), {}).get(codec, {}).get(quality)


    print(settings)

    return settings

In [7]:
get_video_info(testfile)

{'video_codec': 'h264',
 'dimensions': '3840x2160',
 'bitrate_mbps': 104.5,
 'fps': 29.97,
 'duration_str': '0:00:16.249567',
 'size_mb': 212.2}

In [14]:
get_export_settings(testfile)

3840 2160


NameError: name 'codec' is not defined